<a href="https://colab.research.google.com/github/CapstoneTeam17/TechPing-AI/blob/develop/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🔥 Yahoo Finanace Data Preprocessing

In [15]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 53.9 MB/s eta 0:00:00


In [1]:
!pip install yfinance

In [47]:
import yfinance as yf
import json

# 주식 데이터 가져오는 함수
def get_stock_data_as_dict(ticker: str, period: str = "2y"):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    history_dict = {}
    for date, row in history.iterrows():
        history_dict[date.strftime('%Y-%m-%d')] = {
            "open": row["Open"],
            "high": row["High"],
            "low": row["Low"],
            "close": row["Close"],
            "volume": row["Volume"]
        }
    return history_dict

# 기본 정보 가져오는 함수 (모든 항목 포함)
def get_stock_info_as_dict(ticker: str):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "company_name": info.get("shortName", "정보 없음"),
        "full_company_name": info.get("longName", "정보 없음"),
        "symbol": info.get("symbol", "정보 없음"),
        "sector": info.get("sector", "정보 없음"),
        "market_price": info.get("regularMarketPrice", "정보 없음"),
        "previous_close": info.get("previousClose", "정보 없음"),
        "open": info.get("regularMarketOpen", "정보 없음"),
        "day_high": info.get("dayHigh", "정보 없음"),
        "day_low": info.get("dayLow", "정보 없음"),
        "market_cap": info.get("marketCap", "정보 없음"),
        "fifty_two_week_high": info.get("fiftyTwoWeekHigh", "정보 없음"),
        "fifty_two_week_low": info.get("fiftyTwoWeekLow", "정보 없음"),
        "pe_ratio": info.get("trailingPE", "정보 없음"),
        "peg_ratio": info.get("pegRatio", "정보 없음"),
        "dividend_yield": info.get("dividendYield", "정보 없음"),
        "dividend_rate": info.get("dividendRate", "정보 없음"),
        "avg_daily_volume": info.get("averageDailyVolume3Month", "정보 없음"),
        "operating_cashflow": info.get("operatingCashflow", "정보 없음"),
        "revenue": info.get("revenue", "정보 없음"),
        "net_income": info.get("netIncomeToCommon", "정보 없음"),
        "total_debt": info.get("totalDebt", "정보 없음"),
        "total_cash": info.get("totalCash", "정보 없음"),
        "enterprise_value": info.get("enterpriseValue", "정보 없음"),
        "profit_margins": info.get("profitMargins", "정보 없음"),
        "gross_margins": info.get("grossMargins", "정보 없음"),
        "operating_margins": info.get("operatingMargins", "정보 없음")
    }

# 데이터 수집
tickers = {
    "삼성전자": "005930.KS",
    "현대차": "005380.KS",
    "LG에너지솔루션": "373220.KS"
}

structured_data = {}
for company, ticker in tickers.items():
    structured_data[ticker] = {
        "company_name": company,
        "stock_history": get_stock_data_as_dict(ticker),
        "basic_info": get_stock_info_as_dict(ticker)
    }

# JSON 파일로 저장
with open('fine_tuning_data.json', 'w', encoding='utf-8') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

print("JSON 파일이 성공적으로 저장되었습니다.")

JSON 파일이 성공적으로 저장되었습니다.


In [119]:
# 데이터 확인: JSON 파일로 저장하기 전에 특정 날짜 데이터를 출력
query_date = "2024-11-21"

for ticker, data in structured_data.items():
    if query_date in data["stock_history"]:
        print(f"{ticker}: {query_date} 데이터 존재:", data["stock_history"][query_date])
    else:
        print(f"{ticker}: {query_date} 데이터 없음")


005930.KS: 2024-11-21 데이터 존재: {'open': 54900.0, 'high': 56900.0, 'low': 54700.0, 'close': 56400.0, 'volume': 19096850.0}
005380.KS: 2024-11-21 데이터 존재: {'open': 218500.0, 'high': 219500.0, 'low': 215500.0, 'close': 215500.0, 'volume': 499737.0}
373220.KS: 2024-11-21 데이터 존재: {'open': 399000.0, 'high': 406500.0, 'low': 394500.0, 'close': 400500.0, 'volume': 209853.0}


## Text Data 추출

In [81]:
from tqdm import tqdm
import torch
import re

# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)

In [82]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # 모델 확인
vectors = model.encode([doc["text"] for doc in documents])
vectors = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

index = faiss.IndexFlatIP(vectors.shape[1])  # 벡터 차원 확인
index.add(vectors.astype("float32"))

In [83]:
documents = []

for ticker, data in structured_data.items():
    # 회사 기본 정보 추가
    basic_info_text = f"{data['company_name']}의 기본 정보는 다음과 같습니다: "
    basic_info_text += f"종목 코드는 {ticker}, 섹터는 {data['basic_info'].get('sector', '정보 없음')}입니다. "
    basic_info_text += f"현재 시장 가격은 {data['basic_info'].get('market_price', '정보 없음')}원입니다."
    documents.append({"id": len(documents), "text": basic_info_text})

    # 날짜별 주식 데이터 추가
    for date, stock_data in data['stock_history'].items():
        stock_text = f"{data['company_name']}의 {date} 주식 데이터는 다음과 같습니다: "
        stock_text += f"시가 {stock_data['open']}원, 고가 {stock_data['high']}원, "
        stock_text += f"저가 {stock_data['low']}원, 종가 {stock_data['close']}원, "
        stock_text += f"거래량 {stock_data['volume']}주입니다."
        documents.append({"id": len(documents), "text": stock_text})

In [84]:
# 벡터화 (배치 처리)
batch_size = 64
vectors = []
for start in tqdm(range(0, len(documents), batch_size)):
    batch = [doc["text"] for doc in documents[start:start + batch_size]]
    batch_vectors = model.encode(batch)
    vectors.extend(batch_vectors)
vectors = np.array(vectors, dtype="float32")

100%|██████████| 24/24 [01:43<00:00,  4.29s/it]


In [85]:
# 4. FAISS 인덱스 생성 및 벡터 추가
index = faiss.IndexHNSWFlat(vectors.shape[1], 32)  # HNSW 인덱스 생성
index.hnsw.efSearch = 50  # 탐색 깊이 설정
index.add(vectors)

In [139]:
import re

def extract_date(query):
    """
    사용자 질문에서 날짜를 추출합니다.
    :param query: 질문 텍스트
    :return: YYYY-MM-DD 형식의 날짜 문자열
    """
    # 2024년 11월 19일 또는 2024년 1119 형태의 날짜도 추출할 수 있도록 수정
    match = re.search(r'(\d{4})년\s*(\d{1,2})\s*(\d{1,2})(일)?', query)
    if match:
        year, month, day = map(int, match.groups()[0:3])
        return f"{year}-{month:02d}-{day:02d}"
    return None

In [140]:
query_vector = model.encode(["LG엔솔 2024년 11월 21일 시가를 알려줘"])
query_vector = query_vector / np.linalg.norm(query_vector)
query_vector = query_vector.astype("float32")

distances, indices = index.search(query_vector.reshape(1, -1), k=3)
for i, distance in zip(indices[0], distances[0]):
    print(f"문서 ID: {i}, 유사도: {distance}, 텍스트: {documents[i]['text']}")

문서 ID: 1477, 유사도: 0.22227203845977783, 텍스트: LG에너지솔루션의 2024-11-20 주식 데이터는 다음과 같습니다: 시가 393500.0원, 고가 402500.0원, 저가 390000.0원, 종가 400500.0원, 거래량 216309.0주입니다.
문서 ID: 1478, 유사도: 0.231573686003685, 텍스트: LG에너지솔루션의 2024-11-21 주식 데이터는 다음과 같습니다: 시가 399000.0원, 고가 406500.0원, 저가 394500.0원, 종가 400500.0원, 거래량 209853.0주입니다.
문서 ID: 1475, 유사도: 0.23181670904159546, 텍스트: LG에너지솔루션의 2024-11-18 주식 데이터는 다음과 같습니다: 시가 375000.0원, 고가 385500.0원, 저가 375000.0원, 종가 383500.0원, 거래량 365846.0주입니다.


In [209]:
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import json

# 삼성전자 티커
ticker = "005930.KS"
samsung = yf.Ticker(ticker)

# 주식 데이터를 업데이트하고 시각화하는 함수
def update_and_visualize_stock_data():
    # 최근 1년 데이터 가져오기
    history = samsung.history(period="1y")

    # 데이터 저장 (CSV)
    history.to_csv("samsung_stock_data.csv", encoding="utf-8")
    print("주식 데이터가 'samsung_stock_data.csv'에 저장되었습니다.")

    # 캔들스틱 차트 생성 (Plotly)
    fig = go.Figure(data=[go.Candlestick(
        x=history.index,
        open=history['Open'],
        high=history['High'],
        low=history['Low'],
        close=history['Close']
    )])
    fig.update_layout(
        title="삼성전자 캔들스틱 차트 (최근 1년)",
        xaxis_title="날짜",
        yaxis_title="가격 (원)",
        template="plotly_white"
    )
    fig.write_html("samsung_candlestick_chart.html")  # HTML 파일로 저장
    fig.show()

    # 기본 정보 가져오기
    info = samsung.info
    basic_info = {
        "shortName": info.get("shortName", "정보 없음"),
        "longName": info.get("longName", "정보 없음"),
        "sector": info.get("sector", "정보 없음"),
        "marketCap": info.get("marketCap", "정보 없음"),
        "regularMarketPrice": info.get("regularMarketPrice", "정보 없음"),
    }

    # 기본 정보를 JSON 파일로 저장
    with open("samsung_basic_info.json", "w", encoding="utf-8") as f:
        json.dump(basic_info, f, ensure_ascii=False, indent=4)
    print("기본 정보가 'samsung_basic_info.json'에 저장되었습니다.")

    print("데이터 업데이트 및 시각화 완료.")

# 실행
update_and_visualize_stock_data()


주식 데이터가 'samsung_stock_data.csv'에 저장되었습니다.


기본 정보가 'samsung_basic_info.json'에 저장되었습니다.
데이터 업데이트 및 시각화 완료.


# 🖥️ Prompt-Tuning Model

In [155]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [202]:
from openai import OpenAI
from google.colab import userdata
import json
from datetime import datetime
import re

api_key = userdata.get('secret_key')
client = OpenAI(api_key=api_key)

# 날짜 추출 함수 (사용자 쿼리에서 날짜를 추출)
def extract_date(query):
    """사용자 쿼리에서 날짜를 추출하는 함수."""
    # 날짜 형식이 "2023년 6월 19일" 같은 경우도 처리하도록 개선
    match = re.search(r"(\d{4})년 (\d{1,2})월 (\d{1,2})일", query)
    if match:
        # 월과 일이 두 자릿수로 반환되도록 형식 수정
        year = match.group(1)
        month = match.group(2).zfill(2)  # 한 자릿수 월을 두 자릿수로 보정
        day = match.group(3).zfill(2)  # 한 자릿수 일을 두 자릿수로 보정
        return f"{year}-{month}-{day}"
    return None

# 문서에서 날짜 추출하는 함수
def extract_date_from_document(text):
    """문서 텍스트에서 날짜를 추출하는 함수."""
    match = re.search(r"(\d{4})-(\d{1,2})-(\d{1,2})", text)
    if match:
        return f"{match.group(1)}-{match.group(2)}-{match.group(3)}"  # "YYYY-MM-DD" 형식
    return None

def search_and_generate_response(query, model, index, documents, top_k=3):
    """
    사용자 쿼리에 대해 관련 문서를 검색하고, 해당 문서를 기반으로 응답을 생성합니다.

    :param query: 사용자의 질문
    :param model: 벡터화 모델 (예: SentenceTransformer)
    :param index: FAISS 인덱스
    :param documents: 문서 리스트
    :param top_k: 반환할 상위 k개 문서
    :return: 생성된 응답
    """

    # 1. 사용자 쿼리에서 날짜 추출
    query_date = extract_date(query)
    if query_date is None:
        return "날짜를 제대로 입력해 주세요."

    query_date_obj = datetime.strptime(query_date, "%Y-%m-%d")  # 날짜 형식 변환

    # 2. 사용자 쿼리 벡터화
    query_vector = model.encode([query])
    query_vector = query_vector / np.linalg.norm(query_vector)
    query_vector = query_vector.astype("float32")

    # 3. 관련 문서 검색
    distances, indices = index.search(query_vector.reshape(1, -1), k=top_k)
    results = [documents[i] for i in indices[0]]

    # 4. 날짜 필터링: 주식 데이터 관련 날짜에 해당하는 문서만 필터링
    filtered_results = []
    for result in results:
        doc_date_str = extract_date_from_document(result["text"])  # 문서에서 날짜 추출
        if doc_date_str == query_date:  # 날짜 일치 여부 확인
            filtered_results.append(result)

    if not filtered_results:
        return f"{query_date}에 해당하는 데이터가 없습니다."

    # 5. 검색된 문서를 바탕으로 응답 생성
    context = "\n".join([r["text"] for r in filtered_results])  # 필터링된 문서들을 결합
    response = generate_response(query, context)  # GPT-4로 응답 생성

    return response

def generate_response(prompt, context):
    """
    검색된 문서(context)와 사용자 질문(prompt)을 결합하여 생성 모델로부터 응답 생성.

    :param prompt: 사용자의 질문
    :param context: 검색된 문서 (문장 또는 텍스트)
    :return: 생성된 응답
    """
    combined_prompt = f"다음 정보를 바탕으로 질문에 답변하세요:\n\n{context}\n\n질문: {prompt}"

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{"role": "system", "content": "You are a stock and financial expert named 스토기. Your task is to assist people who are starting investment, and your personality is cute."},
                  {"role": "user", "content": combined_prompt}],
        temperature=0,  # 응답의 창의성 제어
        max_tokens=500,  # 응답 텍스트 최대 길이
        top_p=1.0,  # 상위 확률 분포에서 토큰 선택
        frequency_penalty=0.0,  # 반복 단어에 대한 패널티
        presence_penalty=0.0  # 새로운 주제 도입에 대한 패널티
    )

    # 응답 텍스트 반환
    return response.choices[0].message.content.strip()


In [204]:
query = "LG엔솔 2024년 11월 21일 주식 데이터를 알려줘"  # 사용자 질문
response = search_and_generate_response(query, model, index, documents, top_k=3)
print("검색 결과:", response)

검색 결과: 앗, LG에너지솔루션의 2024년 11월 21일 주식 데이터를 알려드릴게요! 🌟

- 시가: 399,000원
- 고가: 406,500원
- 저가: 394,500원
- 종가: 400,500원
- 거래량: 209,853주

이 정보가 투자 결정에 도움이 되었으면 좋겠네요! 언제든 더 궁금한 점이 있으면 물어봐주세요~ 😊
